In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#导入和模块

In [ ]:
import numpy as np
import pandas as pd 
import os
import pathlib
import PIL
from pathlib import Path
from PIL import Image, ImageDraw
from math import sqrt
import ast

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

##训练

In [ ]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
sub = pd.read_csv("../input/tensorflow-great-barrier-reef/example_sample_submission.csv")

path = Path('../input/tensorflow-great-barrier-reef/train_images')
filepaths = list(path.glob(r'**/*.jpg'))

In [ ]:
# checking the train test lengths
print("Number of training samples: ", len(train))
print("Number of testing samples: ", len(test))

In [ ]:
train.head(150)

#看起来很多镜框都没有我们的海星

In [ ]:
# lets see how many frames with no starfishes
train_clean = train.loc[train["annotations"] != "[]"]
print(f"No starfishes in {len(train)-len(train_clean)} samples.")
print(f"The clean train set has {len(train_clean)} images for us to work with.")

In [ ]:
train_clean.head()

##序列分布

In [ ]:
# Checking out the number of sequences
len(train_clean.sequence.value_counts())

In [ ]:
# rows per each sequence
print("Sequence Samples")
print(train_clean.sequence.value_counts())

In [ ]:
seq_df = train_clean.sequence.value_counts().to_frame()
plt.figure(figsize=(16, 9))
sns.barplot(x=seq_df.index, y=list(seq_df.sequence), palette="Greens_d")
plt.title("Distribution of Sequences")
plt.xlabel("Sequence Id")
plt.ylabel("Frequency")
plt.show()

##海星的数量

In [ ]:
num_boxes = []
annotations_clean = []
for elem in train_clean.annotations:
    ann = ast.literal_eval(elem)
    num_boxes.append(len(ann))
    annotations_clean.append(ann)

In [ ]:
# adding num boxes per row and changing the annotations column to a proper python parseable list of dictionaries
train_clean["num_boxes"] = num_boxes
train_clean["annotations"] = annotations_clean

In [ ]:
train_clean.head()

In [ ]:
print("#box Frequency")
print(train_clean.num_boxes.value_counts())

In [ ]:
# number of bounding boxes in the clean train datasets
print(f"Number of Bounding Boxes in the dataset: {train_clean.num_boxes.sum()}")

##边界框数的分布

In [ ]:
box_count = train_clean.num_boxes.value_counts().to_frame()

In [ ]:
plt.figure(figsize=(16, 9))
sns.barplot(x=box_count.index, y=list(box_count.num_boxes), palette="Greens_d")
plt.title("Distribution of Num_boxes")
plt.xlabel("# of Boxes")
plt.ylabel("Frequency")
plt.show()

看起来57%的数据点只有一个边界框，其次是19.1%的数据点有两个边界框

##观察海星

In [ ]:
#structure of a annotation
list(train_clean["annotations"])[0]

In [ ]:
# generating paths for input images
src = '../input/tensorflow-great-barrier-reef/train_images'
paths = []
for row in train_clean.image_id:
    vid_num = row.split('-')[0]
    img_num = row.split('-')[1]
    paths.append(os.path.join(src,f'video_{vid_num}',img_num+'.jpg'))

In [ ]:
train_clean['paths'] = paths

In [ ]:
# classic way of iterating through and drawing the bounding boxes on an image
def vis_boxes(img_path, bboxes):
    coords = []
    for box in bboxes:
        x1 = box['x']
        y1 = box['y']
        x2 = x1 + box['width']
        y2 = y1 + box['height']
        coords.append([x1, y1, x2, y2])
        
    img = Image.open(img_path)
    img1 = img.copy()
    draw = ImageDraw.Draw(img1)
    for elem in coords:
        draw.rectangle(elem, outline='red', width=7)
    
    return img1

In [ ]:
train_clean.head()

##具有最大边界框的序列

In [ ]:
# number of bounding boxes per each sequence
train_clean.groupby('sequence').num_boxes.sum().to_frame()

##看一些例子

In [ ]:
# lets plot a few
# some inspiration from https://www.kaggle.com/sjyangkevin/eda-bounding-box-analysis-annotated-videos

plt.figure(figsize=(16, 9))
n_images = 9
count = 0
r,c = int(sqrt(n_images)), int(sqrt(n_images))
train_plot = train_clean.sample(n = n_images)

for _, row in train_plot.iterrows():
    img_path = row['paths']
    bboxes = row['annotations']
    plt.subplot(r, c, count + 1)
    img_out = vis_boxes(img_path, bboxes)
    plt.imshow(img_out)
    count+=1

plt.show()
plt.tight_layout()